In [1]:
import csv
import pandas as pd
import requests
import re
import shutil
from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import drive

In [2]:
# Function to write data to CSV file
def write_to_csv(data, filename):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write header row
        writer.writerow(['title', 'link', 'date', 'content', 'is_fake'])
        # Write data rows
        for row in data:
            writer.writerow(row)

In [ ]:
# Tempo Fake News Scraping

url = 'https://cekfakta.tempo.co/2024/' # input year 2020

# Initialize list to store data
data = []

for i in range(1, 13):
  if (i < 10):
    url_page = url+'0{}'.format(i)
  else:
    url_page = url+'{}'.format(i)
  page = requests.get(url_page)
  soup = BeautifulSoup(page.text, 'html')

  print(i)

  # Find all news articles
  news = soup.find_all('article', 'text-card')

  for n in news:
    title = n.find('a').get_text()
    link = n.find('a')['href']

    # Split title and category
    title_first_space_index = title.find(',')
    category = title[:title_first_space_index]
    title_news = title[title_first_space_index + 1:].strip()

    if (category == "Benar") and link is not None:
      # Get news contents
      if link.startswith("//"):
        news_link = "http:" + link
      else:
        news_link = link
      get_content = requests.get(news_link)
      soup_content = BeautifulSoup(get_content.text, 'html')
      date = soup_content.find('div', 'detail-title').find('p', 'date margin-bottom-sm').get_text()
      content = soup_content.find('div', 'detail-in').find_all('p')
      paragraph = ''
      for p in content:
        #p.span.unwrap()
        paragraph += p.get_text()

      # News information source
      is_fake = 1

      # Append data to list
      data.append([title_news, news_link, date, paragraph, is_fake])

# File name for the CSV
filename = 'dataset_tempo_benar_2024.csv'

# Write data to CSV file
write_to_csv(data, filename)

print(f"Data has been written to {filename}")

In [12]:
# Read CSV file
df = pd.read_csv('/content/dataset_tempo_benar_2024.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    59 non-null     object
 1   link     59 non-null     object
 2   date     59 non-null     object
 3   content  59 non-null     object
 4   is_fake  59 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 2.4+ KB


,title,link,date,content,is_fake
0,Air Asia Umumkan Larangan Membawa Produk Berba...,https://cekfakta.tempo.co/fakta/2932/benar-air...,"Senin, 3 Juni 2024 19:30 WIB",Sebuah gambar beredar di WhatsApp dan akun Fac...,1
1,Video tentang WNI di Taipei Lapor Surat Suara ...,http://cekfakta.tempo.co/fakta/2725/benar-vide...,"Rabu, 24 Januari 2024 21:00 WIB",Sebuah video beredar di Instagram dan Facebook...,1
2,BPOM Pernah Gerebek Produsen Saus yang Gunakan...,http://cekfakta.tempo.co/fakta/2720/benar-bpom...,"Senin, 22 Januari 2024 15:48 WIB",Sebuah video beredar di WhatsApp dengan klaim ...,1
3,Klaim Muhaimin Iskandar bahwa 16 juta Petani H...,http://cekfakta.tempo.co/fakta/2717/benar-klai...,"Senin, 22 Januari 2024 11:34 WIB","Calon Wakil Presiden nomor urut 1, Muhaimin Is...",1
4,Klaim Mahfud MD Bahwa Food Estate Program Gagal,http://cekfakta.tempo.co/fakta/2712/benar-klai...,"Minggu, 21 Januari 2024 22:04 WIB","Calon Wakil Presiden nomor urut 03, Mahfud MD,...",1


In [13]:
df0 = "dataset_tempo_benar_2020.csv"
df1 = "dataset_tempo_benar_2021.csv"
df2 = "dataset_tempo_benar_2022.csv"
df3 = "dataset_tempo_benar_2023.csv"
df4 = "dataset_tempo_benar_2024.csv"

# List of CSV file names to merge
csv_files = [df0, df1, df2, df3, df4]

# Reads each CSV file and saves it in a list
data_frames = [pd.read_csv(file) for file in csv_files]

# Combines all DataFrames in the list into one DataFrame
df = pd.concat(data_frames, ignore_index=True)

# remove duplicate title content
df = df.drop_duplicates(subset=['title'], keep='first')

df.info()
df

<class 'pandas.core.frame.DataFrame'>
Index: 94 entries, 0 to 145
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    94 non-null     object
 1   link     94 non-null     object
 2   date     94 non-null     object
 3   content  94 non-null     object
 4   is_fake  94 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 4.4+ KB


,title,link,date,content,is_fake
0,Air Asia Umumkan Larangan Membawa Produk Berba...,https://cekfakta.tempo.co/fakta/2932/benar-air...,"Senin, 3 Juni 2024 19:30 WIB",Sebuah gambar beredar di WhatsApp dan akun Fac...,1
16,Klaim Ini Video Tumpukan Jenazah Covid-19 yang...,http://cekfakta.tempo.co/fakta/1347/benar-klai...,"Selasa, 27 April 2021 16:07 WIB",Video yang memperlihatkan kantong-kantong jena...,1
19,Klaim Pria Ini Ditangkap Polisi karena Tak Per...,http://cekfakta.tempo.co/fakta/1424/benar-klai...,"Rabu, 23 Juni 2021 18:53 WIB",Gambar tangkapan layar sebuah unggahan di Inst...,1
23,Eropa Tambahkan Gangguan Saraf Langka Dalam Da...,http://cekfakta.tempo.co/fakta/1504/benar-erop...,"Selasa, 14 September 2021 20:41 WIB",Sebuah unggahan berisi klaim bahwa vaksin Astr...,1
35,"Seorang Pria Italia Terinfeksi Cacar Monyet, C...",http://cekfakta.tempo.co/fakta/1910/benar-seor...,"Rabu, 31 Agustus 2022 13:50 WIB",Sebuah unggahan mengklaim pria asal Italia men...,1
...,...,...,...,...,...
140,Video dengan Narasi Sejarah Keturunan Tionghoa...,http://cekfakta.tempo.co/fakta/2853/benar-vide...,"Selasa, 16 April 2024 20:49 WIB",Sebuah video beredar di WhatsApp yang berisi n...,1
142,Narasi yang Mengatakan Sering Banyak Minum Air...,http://cekfakta.tempo.co/fakta/2907/benar-nara...,"Kamis, 16 Mei 2024 21:20 WIB",Sebuah video beredar di Facebook [arsip] yang ...,1
143,Kasus Demam Berdarah Dengue Tinggi karena Peru...,http://cekfakta.tempo.co/fakta/2900/benar-kasu...,"Selasa, 14 Mei 2024 21:38 WIB",Kementerian Kesehatan menyatakan bahwa tinggin...,1
144,Video Demonstrasi Warga Jepang Terkait Pandemi...,http://cekfakta.tempo.co/fakta/2894/benar-vide...,"Senin, 13 Mei 2024 21:44 WIB",Kolase video yang memperlihatkan aksi demonstr...,1


In [14]:
df.to_csv('dataset-tempo-benar.csv', index=False)

In [16]:
from google.colab import files
files.download("dataset-tempo-benar.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>